In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from scipy.stats import skew, kurtosis
import statsmodels.tsa.vector_ar.vecm as vecm
from statsmodels.tsa.vector_ar.vecm import coint_johansen, VECM
from statsmodels.tsa.stattools import adfuller, acf
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.stats.sandwich_covariance import cov_hac
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tools.sm_exceptions import ValueWarning
from statsmodels.stats.diagnostic import acorr_ljungbox, het_arch, breaks_cusumolsresid
import statsmodels.stats.sandwich_covariance as sw
from statsmodels.stats.sandwich_covariance import cov_hac, weights_bartlett



In [2]:
btc = pd.read_csv('BTCUSDT_filtered.csv')
btc_f = pd.read_csv('BTCUSDT_futures_filtered.csv')
eth = pd.read_csv('ETHUSDT_filtered.csv')
eth_f = pd.read_csv('ETHUSDT_futures_filtered.csv')
bch = pd.read_csv('BCHUSDT_filtered.csv')
bch_f = pd.read_csv('BCHUSDT_futures_filtered.csv')
doge = pd.read_csv('DOGEUSDT_filtered.csv')
doge_f = pd.read_csv('DOGEUSDT_futures_filtered.csv')
spot_data = bch.copy()
futures_data = bch_f.copy()

In [3]:

def describe_returns(data):
    mean = data['daily_return'].mean()
    std_dev = data['daily_return'].std()
    skewness = skew(data['daily_return'].dropna())
    kurt = kurtosis(data['daily_return'].dropna())
    autocorr = data['daily_return'].autocorr()

    return {'mean': mean, 'std_dev': std_dev, 'skewness': skewness, 'kurtosis': kurt, 'autocorrelation': autocorr}

# Selecting optimal lag order using BIC
def select_k_ar_diff(data, maxlags=168, trend='ct'):
    bic_values = []
    for lag in range(1, maxlags + 1):
        model = VAR(data)
        result = model.fit(lag, trend=trend)
        bic_values.append(result.bic)
    
    optimal_lag = bic_values.index(min(bic_values)) + 1
    return optimal_lag

#ADF Test
def adf_test(series):
    result = adfuller(series)
    print(f"ADF Statistic: {result[0]}")
    print(f"p-value: {result[1]}")
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))
    print("")

# Johansen cointegration test
def johansen_test(s1, s2):
    df = pd.concat([s1, s2], axis=1).dropna()
    df.columns = ['spot_log_price', 'futures_log_price']
    k_ar_diff = select_k_ar_diff(df)
    print(f"Optimal lag order (k_ar_diff) based on BIC: {k_ar_diff}")

    result = coint_johansen(df, det_order=1, k_ar_diff=k_ar_diff)
    print("Johansen Cointegration Test:")
    print("Trace Statistic:")
    print(result.lr1)
    print("Critical Values (90%, 95%, 99%):")
    print(result.cvt)
    print("")
    print("Eigen Statistic:")
    print(result.lr2)
    print("Critical Values (90%, 95%, 99%):")
    print(result.cvm)
    print("")
    return k_ar_diff

# Ljung-Box Q test
def ljung_box_test(series, lags=None, significance_level=0.05, title=''):
    result = acorr_ljungbox(series, lags=lags, return_df=True)
    print(f"Ljung-Box Q Test ({title}):")
    print(result)

    autocorrelated_lags = result[result['lb_pvalue'] < significance_level].index
    if autocorrelated_lags.empty:
        print(f"No autocorrelation found up to lag {lags} at significance level {significance_level}")
    else:
        print(f"Autocorrelation found for lags: {autocorrelated_lags.tolist()} at significance level {significance_level}")
    print("")

# Engle's ARCH test
def arch_test(series, lags=None, title=''):
    test_stat, p_value, _, _ = het_arch(series, nlags=lags)
    print(f"Engle's ARCH Test ({title}):")
    print(f"Test Statistic: {test_stat}")
    print(f"P-value: {p_value}")

    significance_level = 0.05
    if p_value < significance_level:
        print(f"Reject the null hypothesis: Conditional heteroscedasticity is present at significance level {significance_level}")
    else:
        print(f"Fail to reject the null hypothesis: No evidence of conditional heteroscedasticity at significance level {significance_level}")
    print("")


In [4]:
# Convert the timestamp to datetime
spot_data['Date'] = pd.to_datetime(spot_data['Date'])
futures_data['Date'] = pd.to_datetime(futures_data['Date'])

# Calculate log prices
spot_data['log_price'] = np.log(spot_data['Close'])
futures_data['log_price'] = np.log(futures_data['Close'])

# Resample hourly data to daily data
spot_daily = spot_data.resample('D', on='Date').last()
futures_daily = futures_data.resample('D', on='Date').last()

# Calculate daily returns
spot_daily['daily_return'] = spot_daily['log_price'].pct_change()
futures_daily['daily_return'] = futures_daily['log_price'].pct_change()
spot_daily = spot_daily.dropna()
futures_daily = futures_daily.dropna()


In [5]:
spot_stats = describe_returns(spot_daily)
futures_stats = describe_returns(futures_daily)

print("Spot Daily Returns Description:", spot_stats)
print("Futures Daily Returns Description:", futures_stats)

Spot Daily Returns Description: {'mean': -5.119457063810327e-05, 'std_dev': 0.009968674413851318, 'skewness': -1.0306033176063698, 'kurtosis': 17.695231131886146, 'autocorrelation': -0.12881661308815087}
Futures Daily Returns Description: {'mean': -5.1011314947859964e-05, 'std_dev': 0.010018773846512513, 'skewness': -1.0697460220440858, 'kurtosis': 18.36050210178791, 'autocorrelation': -0.13123256807628572}


In [6]:
# Test for stationarity
print("ADF Test for Spot Hourly Log Prices:")
adf_test(spot_data['log_price'])
print("\nADF Test for Futures Hourly Log Prices:")
adf_test(futures_data['log_price'])

ADF Test for Spot Hourly Log Prices:
ADF Statistic: -0.9901879821218502
p-value: 0.7568261650773558
	1%: -3.431
	5%: -2.862
	10%: -2.567


ADF Test for Futures Hourly Log Prices:
ADF Statistic: -0.9909182139053128
p-value: 0.7565622449647764
	1%: -3.431
	5%: -2.862
	10%: -2.567



In [7]:
spot_data['log_price_diff'] = spot_data['log_price'].diff()
futures_data['log_price_diff'] = futures_data['log_price'].diff()

spot_data = spot_data.dropna()
futures_data = futures_data.dropna()
print("ADF Test for Spot Hourly Log Prices:")
adf_test(spot_data['log_price_diff'])
print("\nADF Test for Futures Hourly Log Prices:")
adf_test(futures_data['log_price_diff'])




ADF Test for Spot Hourly Log Prices:
ADF Statistic: -24.696728831152274
p-value: 0.0
	1%: -3.431
	5%: -2.862
	10%: -2.567


ADF Test for Futures Hourly Log Prices:
ADF Statistic: -24.774854498674653
p-value: 0.0
	1%: -3.431
	5%: -2.862
	10%: -2.567



In [8]:
warnings.filterwarnings("ignore", category=ValueWarning)
o_lag = johansen_test(spot_data['log_price_diff'], futures_data['log_price_diff'])

Optimal lag order (k_ar_diff) based on BIC: 15
Johansen Cointegration Test:
Trace Statistic:
[5088.3706133  1578.85689524]
Critical Values (90%, 95%, 99%):
[[16.1619 18.3985 23.1485]
 [ 2.7055  3.8415  6.6349]]

Eigen Statistic:
[3509.51371806 1578.85689524]
Critical Values (90%, 95%, 99%):
[[15.0006 17.1481 21.7465]
 [ 2.7055  3.8415  6.6349]]



In [9]:

ljung_box_test(spot_data['log_price_diff'], lags=o_lag, significance_level = 0.01, title="Spot Log-Price Differences")
ljung_box_test(futures_data['log_price_diff'], lags=o_lag, significance_level = 0.01, title="Futures Log-Price Differences")

Ljung-Box Q Test (Spot Log-Price Differences):
       lb_stat     lb_pvalue
1    11.760115  6.051349e-04
2    36.260898  1.336737e-08
3    40.780930  7.277547e-09
4    44.445560  5.184220e-09
5    44.657872  1.702816e-08
6    46.807270  2.044269e-08
7    50.313253  1.253711e-08
8    59.161955  6.804070e-10
9    61.260956  7.656567e-10
10   61.589058  1.811280e-09
11   76.455612  7.120428e-12
12   94.821678  5.717355e-15
13   96.681005  7.272750e-15
14   99.704099  5.403815e-15
15  100.588069  1.009267e-14
Autocorrelation found for lags: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] at significance level 0.01

Ljung-Box Q Test (Futures Log-Price Differences):
       lb_stat     lb_pvalue
1     1.711576  1.907810e-01
2    66.623030  3.411873e-15
3    86.467283  1.256529e-18
4    93.148743  2.820853e-19
5    94.066300  9.386594e-19
6    94.144210  4.166665e-18
7    95.157781  1.075574e-17
8   103.827384  7.033864e-19
9   104.002958  2.432748e-18
10  105.099329  5.172700e-18
11  138.

In [10]:
arch_test(spot_data['log_price_diff'], lags=o_lag, title="Spot Log-Price Differences")
arch_test(futures_data['log_price_diff'], lags=o_lag, title="Futures Log-Price Differences")

Engle's ARCH Test (Spot Log-Price Differences):
Test Statistic: 1779.2093058651367
P-value: 0.0
Reject the null hypothesis: Conditional heteroscedasticity is present at significance level 0.05

Engle's ARCH Test (Futures Log-Price Differences):
Test Statistic: 4924.103996990094
P-value: 0.0
Reject the null hypothesis: Conditional heteroscedasticity is present at significance level 0.05



In [11]:
# Fit VECM model
data = pd.concat([spot_data['log_price_diff'], futures_data['log_price_diff']], axis=1)
data.columns = ['spot', 'future']
vecm_model = vecm.VECM(data, k_ar_diff=o_lag , coint_rank=1, deterministic='coli')
vecm_fit = vecm_model.fit()
vecm_resid = vecm_fit.resid
print(vecm_fit.summary())

Det. terms outside the coint. relation & lagged endog. parameters for equation spot
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       8.263e-07   7.02e-05      0.012      0.991      -0.000       0.000
L1.spot       -1.6084      0.486     -3.310      0.001      -2.561      -0.656
L1.future      0.6368      0.483      1.317      0.188      -0.311       1.584
L2.spot       -1.2738      0.468     -2.721      0.007      -2.191      -0.356
L2.future      0.3381      0.466      0.726      0.468      -0.575       1.251
L3.spot       -0.7624      0.445     -1.712      0.087      -1.635       0.111
L3.future     -0.1200      0.443     -0.271      0.787      -0.989       0.749
L4.spot       -0.4445      0.419     -1.060      0.289      -1.266       0.377
L4.future     -0.3785      0.417     -0.907      0.364      -1.196       0.439
L5.spot       -0.3612      0.391     -0.924    

In [12]:
print("Shape of the residuals:", vecm_resid.shape)
print("First few rows of the residuals:\n", vecm_resid[:5])

Shape of the residuals: (27691, 2)
First few rows of the residuals:
 [[-0.00108446 -0.00067662]
 [-0.00437841 -0.0039543 ]
 [ 0.00602326  0.0061517 ]
 [-0.00380893 -0.00436263]
 [-0.00376066 -0.00363363]]


In [13]:
def newey_west_single_eq(residuals, nlags, kernel='bartlett', use_correction=True):
    nobs = residuals.shape[0]
    gamma0 = np.cov(residuals, bias=True)
    gamma_sum = 0

    if kernel == 'bartlett':
        weights_func = lambda h, nlags: 1 - h / (nlags + 1)
    else:
        raise NotImplementedError(f"Kernel '{kernel}' not implemented")

    for h in range(1, nlags + 1):
        gamma_h = np.cov(residuals[h:], residuals[:-h], bias=True)
        weight = weights_func(h, nlags)
        gamma_sum += weight * (gamma_h + gamma_h)

    nw_cov = gamma0 + gamma_sum

    if use_correction:
        correction = nobs / (nobs - nlags)
        nw_cov *= correction

    return nw_cov

nlags = o_lag
kernel = 'bartlett'
# Extract the residuals for each equation
spot_residuals = vecm_resid[:, 0]
future_residuals = vecm_resid[:, 1]

# Calculate the Newey-West estimator for each equation
nw_cov_spot = newey_west_single_eq(spot_residuals, nlags, kernel=kernel, use_correction=True)
nw_cov_future = newey_west_single_eq(future_residuals, nlags, kernel=kernel, use_correction=True)

# Print the Newey-West robust covariance estimates
print("Newey-West robust covariance estimate for spot equation:", nw_cov_spot)
print("Newey-West robust covariance estimate for future equation:", nw_cov_future)

Newey-West robust covariance estimate for spot equation: [[2.18468239e-03 9.91250059e-05]
 [9.91250059e-05 2.18455143e-03]]
Newey-West robust covariance estimate for future equation: [[0.00224313 0.00010408]
 [0.00010408 0.00224301]]


In [14]:
ljung_box_test(spot_residuals, lags=o_lag, significance_level = 0.01, title="Spot Residuals")

Ljung-Box Q Test (Spot Residuals):
       lb_stat     lb_pvalue
1     0.179127  6.721246e-01
2     1.467731  4.800498e-01
3     3.793046  2.846960e-01
4     7.528707  1.104501e-01
5    15.166137  9.675555e-03
6    27.752384  1.046063e-04
7    42.838161  3.584432e-07
8    61.196993  2.712815e-10
9    86.068358  9.979649e-15
10  114.864300  5.555340e-20
11  145.804905  1.070571e-25
12  185.744292  2.819326e-33
13  233.734151  1.510455e-42
14  290.344883  1.215012e-53
15  357.780415  4.950283e-67
Autocorrelation found for lags: [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] at significance level 0.01



In [15]:
ljung_box_test(future_residuals, lags=o_lag, significance_level = 0.01, title="Future Residuals")

Ljung-Box Q Test (Future Residuals):
       lb_stat     lb_pvalue
1     0.137774  7.105043e-01
2     1.201048  5.485241e-01
3     3.184543  3.640380e-01
4     6.388882  1.719278e-01
5    12.954746  2.380656e-02
6    23.755525  5.791819e-04
7    36.827971  5.056025e-06
8    53.346506  9.241058e-09
9    76.149014  9.369352e-13
10  102.791676  1.503210e-17
11  130.878158  1.156145e-22
12  166.657852  2.301622e-29
13  209.797326  1.322079e-37
14  261.863208  1.005239e-47
15  323.307117  7.872827e-60
Autocorrelation found for lags: [6, 7, 8, 9, 10, 11, 12, 13, 14, 15] at significance level 0.01



In [16]:
o_lag = 4
print(breaks_cusumolsresid(vecm_resid))

(0.3765220283224355, 0.9988934200179092, [(1, 1.63), (5, 1.36), (10, 1.22)])


In [17]:
# Calculate the innovation covariance matrix
omega = np.cov(np.stack([spot_residuals, future_residuals]), bias=True)
# Calculate the innovation correlation matrix
sigma_s = np.std(spot_residuals)
sigma_f = np.std(future_residuals)
rho = omega[0, 1] / (sigma_s * sigma_f)
phi = np.array([[1, rho], [rho, 1]])

# Find the eigenvalues and eigenvectors of the correlation matrix
lamda, G = np.linalg.eig(phi)

# Calculate the matrix V
V = np.diag([sigma_s, sigma_f])

# Calculate the matrix M*
M_star = np.linalg.inv(np.dot(np.dot(G, np.diag(np.sqrt(lamda))), np.dot(G.T, np.linalg.inv(V))))

# Get the error correction coefficient vector
alpha_s, alpha_f = vecm_fit.alpha

# Calculate the MIS for spot and future markets
MIS_s = ((alpha_s * M_star[0, 0] + alpha_f * M_star[1, 0]) ** 2) / ((alpha_s * M_star[0, 0] + alpha_f * M_star[1, 0]) ** 2 + (alpha_s * M_star[0, 1] + alpha_f * M_star[1, 1]) ** 2)
MIS_f = ((alpha_s * M_star[0, 1] + alpha_f * M_star[1, 1]) ** 2) / ((alpha_s * M_star[0, 0] + alpha_f * M_star[1, 0]) ** 2 + (alpha_s * M_star[0, 1] + alpha_f * M_star[1, 1]) ** 2)

# Print the MIS for spot and future markets
print("MIS for spot market:", MIS_s)
print("MIS for future market:", MIS_f)

MIS for spot market: [0.39000737]
MIS for future market: [0.60999263]
